# modeling11-2-nli-binaryclassifier-FiDEncoder-prediction
- making prediction script for binary_classifier_predict.py

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='1'

In [2]:
from pprint import pprint

In [3]:
from accelerate import Accelerator
from accelerate.logging import get_logger

from FiD.src.model import FiDT5
from src.model import FiDEncoderForSequenceClassification

from tqdm.auto import tqdm
from src.data import (
    BinaryCustomDatasetShuffle,
    BinarySentenceDataset,
    BinaryCustomDatasetDecisiveBinaryGold,
)

import logging
import json
import math
import os
import evaluate
from util import utils

import transformers
import torch
import numpy as np
import random
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoModelForSequenceClassification,
    AutoModel,
    AutoConfig,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    set_seed,
    get_scheduler,
)
from util.arguments import ModelArguments, DataTrainingArguments, CustomTrainingArguments
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

NEW_LINE = "\n"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

DATASET_MAPPING = {
    "BinaryCustomDatasetShuffle" : BinaryCustomDatasetShuffle,
    "BinarySentenceDataset" : BinarySentenceDataset,
    'BinaryCustomDatasetDecisiveBinaryGold' : BinaryCustomDatasetDecisiveBinaryGold,
}

In [4]:
# #!/bin/bash

# #echo $SLURM_ARRAY_TASK_ID
# #export i=$SLURM_ARRAY_TASK_ID
# #echo $CUDA_VISIBLE_DEVICES
# export gpu_=$CUDA_VISIBLE_DEVICES

# # decisive_binary_gold_data
# CUDA_VISIBLE_DEVICES="$gpu_" python binary_classifier_predict.py \
# --do_train False \
# --do_eval True \
# --do_predict True \
# --per_device_eval_batch_size 128 \
# --prediction_model_name_or_path /data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/roberta-decisive_binary_gold_data_trial1 \
# --prediction_model_step 380 \
# --max_seq_length 200 \
# --eval_file /data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold/1/decisive_binary_gold_data/binary_decisive_gold_ctx100id_split_dev_1.json \
# --dataset_class BinaryCustomDatasetDecisiveBinaryGold \
# --num_labels 2 \


In [5]:
parser = HfArgumentParser(
    (ModelArguments, DataTrainingArguments, CustomTrainingArguments)
)
model_args, data_args, train_args = parser.parse_args_into_dataclasses([])
# os.environ['WANDB_PROJECT'] = model_args.wandb_project

In [6]:
pprint(vars(model_args))

{'config_base_path': '/data/philhoon-relevance/FiD/pretrained_models/nq_reader_large',
 'config_name': None,
 'git_tag': 'v1.1',
 'max_seq_length': 200,
 'model_architecture': 'roberta-large',
 'model_name_or_path': 'roberta-large',
 'prediction_model_name_or_path': '/data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/roberta-decisive_binary_gold_data_trial1',
 'prediction_model_step': '380',
 'tokenizer_name': None}


In [7]:
pprint(vars(data_args))

{'data': 'NQ-DEV-DPR/5-fold/1',
 'dataset_class': 'BinaryCustomDatasetShuffle',
 'eval_file': '/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold/1/binary_data/binary_ex_ctx100id_split_train_1_partial.json',
 'intact_eval': False,
 'num_labels': 2,
 'overwrite_cache': False,
 'pad_to_max_length': False,
 'train_file': '/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold/1/binary_data/binary_ex_ctx100id_split_train_1_partial.json'}


In [8]:
pprint(vars(train_args))

{'adam_beta1': 0.9,
 'adam_beta2': 0.999,
 'adam_epsilon': 1e-08,
 'best_metric': 'accuracy',
 'checkpointing_steps': None,
 'class_weights': False,
 'do_eval': True,
 'do_predict': False,
 'do_train': True,
 'drop_out_rate': None,
 'gradient_accumulation_steps': 1,
 'headtype': 'adaptive',
 'learning_rate': 5e-05,
 'lr_scheduler_type': 'linear',
 'max_train_steps': None,
 'num_layers': None,
 'num_train_epochs': 3,
 'num_warmup_steps': 0,
 'output_dir': '/data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/testing-scratch',
 'padding': -100,
 'per_device_eval_batch_size': 8,
 'per_device_train_batch_size': 8,
 'report_to': 'wandb',
 'run_name': 'testing_from_scratch',
 'save_max_limit': 3,
 'seed': 42,
 'train_loss_steps': 10,
 'wandb_project': 'binary_classifier',
 'weight_decay': 0.0,
 'with_tracking': True}


In [ ]:
train_args.do_train = False
train_args.do_eval = True
train_args.do_predict = True
train_args.per_device_eval_batch_size = 128

In [ ]:
model_args.prediction_model_name_or_path = '/data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/fid-encoder-linear-decisive-trial6'
model_args.prediction_model_step = '80'
model_args.max_seq_length = 200
model_args.num_labels = 2
train_args.headtype = 'linear'

In [ ]:
# data_args.intact_eval = False
# data_args.eval_file = '/data/philhoon-relevance/binary-classification/NQ-TEST-DPR/binary_decisive_format_ctx100id_test.json'

# data_args.intact_eval = True
# data_args.eval_file = '/data/philhoon-relevance/binary-classification/NQ-TEST-DPR/binary_decisive_gold_ctx100id_test.json'

data_args.dataset_class = 'BinaryCustomDatasetShuffle'

In [ ]:
logger = get_logger(__name__)
accelerator = Accelerator()

In [ ]:
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
    )

In [ ]:
logger.info(accelerator.state, main_process_only=False)
if accelerator.is_local_main_process:
    transformers.utils.logging.set_verbosity_info()
else:
    transformers.utils.logging.set_verbosity_error()

In [ ]:
if not data_args.intact_eval:
    train_args.output_dir = os.path.join(model_args.prediction_model_name_or_path,
                                         f'step_{model_args.prediction_model_step}', 'partial_prediction')
else:
    train_args.output_dir = os.path.join(model_args.prediction_model_name_or_path,
                                         f'step_{model_args.prediction_model_step}', 'intact_prediction')

In [ ]:
print(train_args.output_dir)

In [ ]:
if accelerator.is_main_process and train_args.output_dir is not None:
    os.makedirs(train_args.output_dir, exist_ok=True)
accelerator.wait_for_everyone()

In [ ]:
config_path = model_args.config_base_path

In [ ]:
config_path

In [ ]:
config = AutoConfig.from_pretrained(config_path, num_labels=data_args.num_labels)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('t5-base')

In [ ]:
# pprint(vars(tokenizer))

In [ ]:
model_args.prediction_model_name_or_path

In [ ]:
config_path

In [ ]:
fid_model = FiDT5.from_pretrained(config_path)

In [ ]:
fid_encoder = fid_model.encoder.encoder

In [ ]:
train_args.headtype

In [ ]:
model = FiDEncoderForSequenceClassification(config, fid_encoder, pooler = train_args.headtype)

In [ ]:
pytorch_model_path = os.path.join(model_args.prediction_model_name_or_path,
                                      f'step_{model_args.prediction_model_step}', 'pytorch_model.bin')

In [ ]:
pytorch_model_path

In [ ]:
state_dict_ = torch.load(pytorch_model_path)

In [ ]:
model.load_state_dict(state_dict_)

In [ ]:
# data_args.dataset_class
data_args.eval_file

In [ ]:
eval_file = data_args.eval_file
eval_data = utils.open_json(eval_file)
DataSetClass = DATASET_MAPPING[data_args.dataset_class]
eval_dataset = DataSetClass(eval_data, tokenizer=tokenizer,
                            max_length=model_args.max_seq_length, shuffle=False)


In [ ]:
# for index in random.sample(range(len(eval_dataset)), 5):
#     logger.info(f"Sample {index} of the eval set: {eval_dataset[index]}.")

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer, pad_to_multiple_of=8)

In [ ]:
eval_dataloader = DataLoader(eval_dataset,
                             shuffle=False,
                             collate_fn=data_collator,
                             batch_size=train_args.per_device_eval_batch_size,
                             )

In [ ]:
model, eval_dataloader = accelerator.prepare(
    model, eval_dataloader
)

In [ ]:
metric_acc = evaluate.load("accuracy")
metric_pre = evaluate.load('precision')
metric_re = evaluate.load('recall')
metric_f1 = evaluate.load('f1')

In [ ]:
logger.info("***** Running evaluation *****")
logger.info(f"  Num examples = {len(eval_dataset)}")
logger.info(f"  Instantaneous batch size per device = {train_args.per_device_eval_batch_size}")
logger.info(f"  Steps = {math.ceil(len(eval_dataset)/train_args.per_device_eval_batch_size) + 1}")


In [ ]:
# Saving model_args, data_args, train_args
train_dict = vars(train_args)
logger.info(f"  Saving training_args = {train_dict}")
with open(os.path.join(train_args.output_dir, "train_args.json"), "w") as f:
    json.dump(train_dict, f)

model_dict = vars(model_args)
logger.info(f"  Saving model_args = {model_dict}")
with open(os.path.join(train_args.output_dir, "model_args.json"), "w") as f:
    json.dump(model_dict, f)

data_dict = vars(data_args)
logger.info(f"  Saving data_args = {data_dict}")
with open(os.path.join(train_args.output_dir, "data_args.json"), "w") as f:
    json.dump(data_dict, f)

In [ ]:
eval_progress_bar = tqdm(range(len(eval_dataloader)), disable=not accelerator.is_local_main_process)

In [ ]:
eval_loss = 0
model.eval()
samples_seen = 0
prediction_lst = []
reference_lst = []

In [ ]:
for step, batch in enumerate(eval_dataloader):
    with torch.no_grad():
        outputs = model(**batch)
        loss = outputs.loss
    if train_args.with_tracking:
        eval_loss += loss.detach().float()

    predictions = outputs.logits.argmax(dim=-1)
    predictions, references = accelerator.gather((predictions, batch["labels"]))
    # If we are in a multiprocess environment, the last batch has duplicates
    if accelerator.num_processes > 1:
        if step == len(eval_dataloader) - 1:
            predictions = predictions[: len(eval_dataloader.dataset) - samples_seen]
            references = references[: len(eval_dataloader.dataset) - samples_seen]
        else:
            samples_seen += references.shape[0]
    
    metric_acc.add_batch(
        predictions=predictions,
        references=references,
    )
    metric_pre.add_batch(
        predictions=predictions,
        references=references,
    )
    metric_re.add_batch(
        predictions=predictions,
        references=references,
    )
    metric_f1.add_batch(
        predictions=predictions,
        references=references,
    )
    eval_progress_bar.update(1)
    prediction_lst.extend(predictions.detach().cpu().tolist())
    reference_lst.extend(references.detach().cpu().tolist())
#     print(f'predictions : {predictions}')
#     print(f'prediction_lst : {prediction_lst}')
#     print(f'references : {references}')
#     print(f'eval_loss : {eval_loss}')


In [ ]:
eval_metric = metric_acc.compute()
eval_metric_pre = metric_pre.compute()
eval_metric_re = metric_re.compute()
eval_metric_f1 = metric_f1.compute()

logger.info(f"Accuracy : {eval_metric['accuracy']} Precision : {eval_metric_pre['precision']}")
logger.info(f"Recall : {eval_metric_re['recall']} F1 : {eval_metric_f1['f1']}")
logger.info(f"Eval_loss : {eval_loss.item() / len(eval_dataloader)}")


In [ ]:
result_log = {
    "eval_accuracy": eval_metric['accuracy'],
    "eval_precision": eval_metric_pre['precision'],
    "eval_recall": eval_metric_re['recall'],
    "eval_f1": eval_metric_f1['f1'],
    "eval_loss": eval_loss.item() / len(eval_dataloader),
}

In [ ]:
output_result_path = os.path.join(train_args.output_dir, 'result.json')
with open(output_result_path, "w") as f:
    json.dump(result_log, f)

In [ ]:
output_result_path

In [ ]:
prediction_np = np.array(prediction_lst)
reference_np = np.array(reference_lst)

In [ ]:
for ins, p_, r_ in zip(eval_data, prediction_np, reference_np):
    if str(r_) != ins['em']:
        logger.info(f"Not Matching Instance")
    ins['binary_inference'] = str(p_)

In [ ]:
predcition_output_path = os.path.join(train_args.output_dir, 'prediction.json')
with open(predcition_output_path, "w") as f:
    json.dump(eval_data, f)

In [ ]:
predcition_output_path

# Get top model

In [ ]:
path = '/data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/roberta-decisive_binary_data-weighted-trial3_desktop1'
path = '/data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/roberta-decisive_binary_gold_data_trial1'








In [ ]:
import pathlib
from pprint import pprint

In [ ]:
path

In [ ]:
path = pathlib.Path(path)

In [ ]:
files = path.glob('*/*[0-9]_results.json')
# pprint(list(files))

In [ ]:
step_result = []

for file in files:
    step = str(file).split('/')[-2]
    result = utils.open_json(file)
    step_result.append(result)
    
pprint(step_result)

In [ ]:
score_dict = {}
for dict_ins in step_result:
    score_dict[dict_ins['step']] = 0
print(score_dict)

In [ ]:
top_n = len(step_result)

In [ ]:
sorted(step_result, key=lambda x: x['eval_accuracy'], reverse = True) 

In [ ]:
def sort_by(result, key, top_k, score_dict):
    newlist = sorted(result, key=lambda d: d[key], reverse = True) 
    print(f'sorting by {key}')
    for dic_ in newlist[:top_k]:
        print(f"step : {dic_['step']}, key : {dic_[key]}")
    for dic_, score_ in zip(newlist, list(range(len(step_result) ,0 , -1))):
        score_dict[dic_['step']] += score_
    print(score_dict)

In [ ]:
sort_by(step_result, 'eval_accuracy', top_n, score_dict)

In [ ]:
sort_by(step_result, 'eval_f1', top_n, score_dict)

In [ ]:
sort_by(step_result, 'eval_precision', top_n, score_dict)

In [ ]:
sort_by(step_result, 'eval_recall', top_n, score_dict)

In [ ]:
def sort_by(result, key, top_k, score_dict):
    newlist = sorted(result, key=lambda d: d[key], reverse = True) 
    print(f'sorting by {key}')
    for dic_ in newlist[:top_k]:
        print(f"step : {dic_['step']}, key : {dic_[key]}")
    for dic_, score_ in zip(newlist, list(range(len(step_result) ,0 , -1))):
        score_dict[dic_['step']] += score_
    print(score_dict)

In [ ]:
sorted_score_dict = sorted(score_dict.items(), key=lambda x:x[1], reverse = True)
for k, v in sorted_score_dict:
    print(f'step {k} : scores : {v}')

## Reverse REsult

In [ ]:
files = path.glob('*/*rev_results.json')
# pprint(list(files))

In [ ]:
step_result = []

for file in files:
    step = str(file).split('/')[-2]
    result = utils.open_json(file)
    step_result.append(result)

In [ ]:
# pprint(step_result)

In [ ]:
score_dict = {}
for dict_ins in step_result:
    score_dict[dict_ins['step']] = 0
print(score_dict)

In [ ]:
top_n = len(step_result)

In [ ]:
sort_by(step_result, 'eval_rev_accuracy', top_n, score_dict)

In [ ]:
sort_by(step_result, 'eval_rev_precision', top_n, score_dict)

In [ ]:
sort_by(step_result, 'eval_rev_recall', top_n, score_dict)

In [ ]:
sort_by(step_result, 'eval_rev_f1', top_n, score_dict)

In [ ]:
def sort_by(result, key, top_k, score_dict):
    newlist = sorted(result, key=lambda d: d[key], reverse = True) 
    print(f'sorting by {key}')
    for dic_ in newlist[:top_k]:
        print(f"step : {dic_['step']}, key : {dic_[key]}")
    for dic_, score_ in zip(newlist, list(range(len(step_result) ,0 , -1))):
        score_dict[dic_['step']] += score_
    print(score_dict)

In [ ]:
sorted_score_dict = sorted(score_dict.items(), key=lambda x:x[1], reverse = True)
for k, v in sorted_score_dict:
    print(f'step {k} : scores : {v}')

In [ ]:
prediction_file = '/data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/roberta-decisive_binary_gold_data_trial1/step_320/prediction/prediction.json'
# prediction_file = '/data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/fid-encoder-adpative-decisive-trial7/step_360/partial_prediction/prediction.json'
predict_data = utils.open_json(prediction_file)


In [ ]:
len(predict_data)

In [ ]:
predict_data[0]

In [ ]:
ref_lst = []
pre_lst = []
for ins in predict_data:
    ref_lst.append(int(ins['em']))
    pre_lst.append(int(ins['binary_inference']))

In [ ]:
from collections import Counter

In [ ]:
def check_ratio(data):
    counter = Counter(data)
    print(counter)

In [ ]:
check_ratio(ref_lst)

In [ ]:
check_ratio(pre_lst)

In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd

In [ ]:
reference_np = np.array(ref_lst)
prediction_np = np.array(pre_lst)
y_actu = pd.Series(reference_np, name='Actual')
y_pred = pd.Series(prediction_np, name='Predicted')

In [ ]:
df_confusion = pd.crosstab(y_actu, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)
df_confusion

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [ ]:
accuracy = accuracy_score(y_actu, y_pred)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_actu, y_pred)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_actu, y_pred)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_actu, y_pred)
print('F1 score: %f' % f1)

In [ ]:
reversey_pred = y_pred.map(lambda x : 0 if x == 1 else 1)

In [ ]:
reversey_pred

In [ ]:
reversey_actu = y_actu.map(lambda x : 0 if x == 1 else 1)

In [ ]:
accuracy = accuracy_score(reversey_actu, reversey_pred)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(reversey_actu, reversey_pred)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(reversey_actu, reversey_pred)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(reversey_actu, reversey_pred)
print('F1 score: %f' % f1)

## Reversing Result

In [ ]:
y_actu = pd.Series(reference_np, name='Actual')
y_pred = pd.Series(prediction_np, name='Predicted')
df_confusion = pd.crosstab(y_actu, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)
df_confusion

In [ ]:
len(eval_data)

In [ ]:
len(prediction_np)

In [ ]:
for ins, p_, r_ in zip(eval_data, prediction_np, reference_np):
    if str(r_) != ins['em']:
        logger.info(f"Not Matching Instance")
        break
    ins['binary_inference'] = str(p_)

In [ ]:
eval_data[2]

In [ ]:
predcition_output_path = os.path.join(train_args.output_dir, 'prediction.json')

In [ ]:
with open(predcition_output_path, "w") as f:
    json.dump(eval_data, f)